In [1]:
import sys
import os,glob
import re, numpy as np, pandas as pd
from pprint import pprint
from scipy.spatial import distance
import matplotlib.pyplot as plt

In [2]:
def toINT(text):
    if text[-1]=='M':
        return float(text.replace('M',''))*10**6
    if text[-1]=='B':
        return float(text.replace('B',''))*10**9
    if text[-1]=='k':
        return float(text.replace('k',''))*10**3

# Import vectors

In [3]:
vector = pd.read_csv('/Users/hellofutrue/Desktop/Insight/Python/Feb/files/vector.csv')
vector = vector.set_index('Unnamed: 0') #index = influencer's name

#import outside fanbase info
f = open('/Users/hellofutrue/Desktop/Insight/Python/Feb/files/growing_fashion_influencer_list.txt', 'r')
list = f.readlines()
f.close()
candid_list = list[0].split()

#top influencers list
w = open('/Users/hellofutrue/Desktop/Insight/Python/Feb/files/top_influencer_list.txt', 'r')
list = w.readlines()
w.close()
top_list = list[0].split()

#import outside database
database = pd.read_csv('/Users/hellofutrue/Desktop/Insight/Python/Untitled Folder/Full_inluencer_list.csv')
database = database.drop_duplicates(keep='last')
database["fanbase"] = database["fanbase"].apply(lambda x: toINT(x))

In [4]:
#import outside fanbase info
f = open('/Users/hellofutrue/Desktop/Insight/Python/Feb/files/growing_fashion_influencer_list.txt', 'r')
list = f.readlines()
f.close()
cadid_list = list[0].split()

# Experimentation

In [39]:
p20 = np.percentile(dst['Euclidean Distance'], 20) #extremely similar (smallest 20%)
p50 = np.percentile(dst['Euclidean Distance'], 50) #similar (smallest 50%)

In [40]:
result = dst.sort_values(['Euclidean Distance'], ascending=[1])[1:6]
result

,Euclidean Distance
saraunderwood,0.0
wolfiecindy,0.0
jccaylen,0.0
winnieharlow,0.0
drayamichele,0.0


In [44]:
result['Similarity mark'] = 'Average'
result

,Euclidean Distance,Similarity mark
saraunderwood,0.0,Average
wolfiecindy,0.0,Average
jccaylen,0.0,Average
winnieharlow,0.0,Average
drayamichele,0.0,Average


In [45]:
for i in range(len(result['Euclidean Distance'])):
    if result['Euclidean Distance'][i] < p50:
        result['Similarity mark'][i] = 'Similar'
    if result['Euclidean Distance'][i] < p20:
        result['Similarity mark'][i] = 'Extremely similar'

/Users/hellofutrue/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/hellofutrue/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [46]:
result

,Euclidean Distance,Similarity mark
saraunderwood,0.0,Extremely similar
wolfiecindy,0.0,Extremely similar
jccaylen,0.0,Extremely similar
winnieharlow,0.0,Extremely similar
drayamichele,0.0,Extremely similar


# Function

In [5]:
def outputtable1(nameinput):
    dst = pd.DataFrame()
    for people in cadid_list:
        row = vector.loc[nameinput, : ].values
        other = vector.loc[people, : ].values
        score = pd.DataFrame(distance.euclidean(row, other),index=[people], columns=['Euclidean Distance'])
        dst = dst.append(score)
    result = dst.sort_values(['Euclidean Distance'], ascending=[1])[1:6]
    #for name in result.index:
    #    result.set_value(name, 'fanbase',database.loc[database['name'] == name]['fanbase'])
    result = result.assign(Similarity_Index = result.iloc[:,0])
    result.loc[result['Similarity_Index'] <= 0.001, 'Similarity_Index'] = 'Extremely similar' 
    return result

In [6]:
outputtable1('jamescharles')

,Euclidean Distance,Similarity_Index
rickeythompson,0.012132,0.0121321
antoniiogarza,0.012137,0.012137
nashgrier,0.014988,0.0149879
joynerlucas,0.014988,0.0149883
joiechavis,0.014988,0.0149884


In [7]:
outputtable1('kendalljenner')

,Euclidean Distance,Similarity_Index
saraunderwood,0.0,Extremely similar
wolfiecindy,0.0,Extremely similar
jccaylen,0.0,Extremely similar
winnieharlow,0.0,Extremely similar
drayamichele,0.0,Extremely similar


In [28]:
def outputtable(nameinput):
    dst = pd.DataFrame()
    for people in cadid_list:
        row = vector.loc[nameinput, : ].values
        other = vector.loc[people, : ].values
        score = pd.DataFrame(distance.euclidean(row, other),index=[people], columns=['Euclidean Distance'])
        dst = dst.append(score)
    result = dst.sort_values(['Euclidean Distance'], ascending=[1])[1:6]
    #for name in result.index:
    #    result.set_value(name, 'fanbase',database.loc[database['name'] == name]['fanbase'])
    result = result.assign(Similarity_Index = result.iloc[:,0])
    result.loc[result['Similarity_Index'] <= 0.001, 'Similarity_Index'] = 'Extremely similar' 
    li = []
    n = len(result)
    for i in range(n):
        li.append([result.index[i],str(result.iloc[i, 0]), result.iloc[i, 1]])
    return li

In [84]:
def predictreach(nameinput):
    result = outputtable(nameinput)
    for candid in outputtable(nameinput).index.values.tolist():
        result.at[candid, 'fanbase'] = database.loc[database['name'] == candid]['fanbase'].values.tolist()[0]
    fanbase_lowerbound = max(result['fanbase'])
    fanbase_upperbound = sum(result['fanbase'])
    return print('The estimated fan base of all 5 suggested ambassadors ranges from', fanbase_lowerbound,'to',fanbase_upperbound)

In [85]:
predictreach('nickiminaj')

The estimated fan base of all 5 suggested ambassadors ranges from 9200000.0 to 30500000.0


In [ ]:
topic_name = ['Topic 1','Topic 2','Topic 3','Topic 4','Topic 5','Topic 6','Topic 7','Topic 8','Topic 9','Topic 10']

In [ ]:
for name in outputtable(nameinput).index.values.tolist():
    y_pos = vector.loc[name,:]
    bars = topic_name
    plt.bar(bars,y_pos,color = (0.5,0.1,0.5,0.6))
    # Create names on the x-axis
    plt.xticks(bars, color='orange')
    plt.yticks(color='orange')
    plt.ylim(0,1) # Limits for the Y axis
    plt.title(name)
    
    plt.show()

In [15]:
resultname = []
for i in range(5):
    name = outputtable('nickiminaj', candid_list,vector)[i][0]
    resultname.append(name)

In [16]:
resultname

['sophiet', 'virginia', 'normani', 'sofiavergara', 'jordynwoods']

In [20]:
def predictreach(nameinput):
    dst = pd.DataFrame()
    for people in candid_list:
        row = vector.loc[nameinput, : ].values
        other = vector.loc[people, : ].values
        score = pd.DataFrame(distance.euclidean(row, other),index=[people], columns=['Euclidean Distance'])
        dst = dst.append(score)
    result = dst.sort_values(['Euclidean Distance'], ascending=[1])[1:6]
    result = result.assign(Similarity_Index = result.iloc[:,0])
    for candid in result.index:
        result.at[candid, 'fanbase'] = database.loc[database['name'] == candid]['fanbase'].values.tolist()[0]
    fanbase_lowerbound = max(result['fanbase'])
    fanbase_upperbound = sum(result['fanbase'])
    return print('The estimated fan base of all 5 suggested ambassadors ranges from', fanbase_lowerbound,'to',fanbase_upperbound)

In [23]:
def predictreach(nameinput):
    dst = pd.DataFrame()
    for people in candid_list:
        row = vector.loc[nameinput, : ].values
        other = vector.loc[people, : ].values
        score = pd.DataFrame(distance.euclidean(row, other),index=[people], columns=['Euclidean Distance'])
        dst = dst.append(score)
    result = dst.sort_values(['Euclidean Distance'], ascending=[1])[1:6]
    result = result.assign(Similarity_Index = result.iloc[:,0])
    for candid in result.index:
        result.at[candid, 'fanbase'] = database.loc[database['name'] == candid]['fanbase'].values.tolist()[0]
    fanbase_lowerbound = max(result['fanbase'])
    fanbase_upperbound = sum(result['fanbase'])
    return fanbase_lowerbound, fanbase_upperbound

# Function actually in use

In [ ]:
def outputtable(nameinput):
    dst = pd.DataFrame()
    for people in cadid_list:
        row = vector.loc[nameinput, : ].values
        other = vector.loc[people, : ].values
        score = pd.DataFrame(distance.euclidean(row, other),index=[people], columns=['Euclidean Distance'])
        dst = dst.append(score)
    result = dst.sort_values(['Euclidean Distance'], ascending=[1])[1:6]
    #for name in result.index:
    #    result.set_value(name, 'fanbase',database.loc[database['name'] == name]['fanbase'])
    result = result.assign(Similarity_Index = result.iloc[:,0])
    result.loc[result['Similarity_Index'] <= 0.001, 'Similarity_Index'] = 'Extremely similar' 
    li = []
    n = len(result)
    for i in range(n):
        li.append([result.index[i],str(result.iloc[i, 0]), result.iloc[i, 1]])
    return li

In [54]:
def outputtable(nameinput):
    dst = pd.DataFrame()
    for people in cadid_list:
        row = vector.loc[nameinput, : ].values
        other = vector.loc[people, : ].values
        score = pd.DataFrame(distance.euclidean(row, other),index=[people], columns=['Euclidean Distance'])
        dst = dst.append(score)
    result = dst.sort_values(['Euclidean Distance'], ascending=[1])[1:6]
    #cuttoff score for marking
    p3 = np.percentile(dst['Euclidean Distance'], 20) #extremely similar (smallest 20%)
    p2 = np.percentile(dst['Euclidean Distance'], 50) #similar (smallest 50%)
    result['Similarity mark'] = 'Average'
    for i in range(len(result['Euclidean Distance'])):
        if result['Euclidean Distance'][i] < p50:
            result['Similarity mark'][i] = 'Similar'
        if result['Euclidean Distance'][i] < p20:
            result['Similarity mark'][i] = 'Extremely similar'
    li = []
    n = len(result)
    for i in range(n):
        li.append([result.index[i],str(round(result.iloc[i, 0],4)), result.iloc[i, 1]])
    return li

In [55]:
outputtable(nameinput)

/Users/hellofutrue/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/hellofutrue/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[['saraunderwood', '0.0', 'Extremely similar'],
 ['wolfiecindy', '0.0', 'Extremely similar'],
 ['jccaylen', '0.0', 'Extremely similar'],
 ['winnieharlow', '0.0', 'Extremely similar'],
 ['drayamichele', '0.0', 'Extremely similar']]

In [ ]:
def predictreach(nameinput):
    dst = pd.DataFrame()
    for people in candid_list:
        row = vector.loc[nameinput, : ].values
        other = vector.loc[people, : ].values
        score = pd.DataFrame(distance.euclidean(row, other),index=[people], columns=['Euclidean Distance'])
        dst = dst.append(score)
    result = dst.sort_values(['Euclidean Distance'], ascending=[1])[1:6]
    result = result.assign(Similarity_Index = result.iloc[:,0])
    for candid in result.index:
        result.at[candid, 'fanbase'] = database.loc[database['name'] == candid]['fanbase'].values.tolist()[0]
    fanbase_lowerbound = max(result['fanbase'])
    fanbase_upperbound = sum(result['fanbase'])
    return fanbase_lowerbound, fanbase_upperbound